In [3]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Food is vital to our survival and keeps our bodies in a functioning state. Food becomes a necessary component of our body’s functioning the moment we are born",
        metadata={"source": "doc1", "author": "Alice", "date": "2023-01-01"}
    ),
    Document(
        page_content="A balanced diet improves our ability to fend off sickness, boosts our immunity, gives us energy throughout the day, and controls our mood. It further aids in meeting our body’s developmental benchmarks at various growth stages.",
        metadata={"source": "doc2", "author": "Bob", "date": "2023-02-01"}
    ),
    Document(
        page_content="Furthermore, food plays a significant role in fostering global cultural experiences and connections with a diverse range of individuals.",
        metadata={"source": "doc3", "author": "Charlie", "date": "2023-03-01"}
    ),
    Document(
        page_content="Cats are one of the most popular and cherished pets worldwide. Known for their playful nature, independence, and affectionate demeanour, cats have been human companions for centuries. ",
        metadata={"source": "doc4", "author": "Alice", "date": "2023-04-01"}
    ),
    Document(
        page_content="The cat is a domestic animal. Its scientific name is Felis catus. It is a small animal that belongs to the “Felidae” family.",
        metadata={"source": "doc5", "author": "Bob", "date": "2023-05-01"})
]



In [4]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.getenv("HUGGINGFACEHUB_API_TOKEN")

llm = ChatGroq(
    model="Llama3-8b-8192",
    groq_api_key=groq_api_key)

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2")

/opt/miniconda3/envs/practice/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
### vectorstores
from langchain_chroma import Chroma
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings)
vectorstore

In [7]:
vectorstore.similarity_search("What is the dog")

[Document(id='ad6ac9d9-a0bc-4d83-98dc-3fb9342c3f38', metadata={'date': '2023-05-01', 'source': 'doc5', 'author': 'Bob'}, page_content='The cat is a domestic animal. Its scientific name is Felis catus. It is a small animal that belongs to the “Felidae” family.'),
 Document(id='504e67b6-828f-4d0e-88b0-f03d1a8daa95', metadata={'source': 'doc4', 'date': '2023-04-01', 'author': 'Alice'}, page_content='Cats are one of the most popular and cherished pets worldwide. Known for their playful nature, independence, and affectionate demeanour, cats have been human companions for centuries. '),
 Document(id='bf38f0ce-a7ef-40de-a042-6291814a17d2', metadata={'date': '2023-03-01', 'source': 'doc3', 'author': 'Charlie'}, page_content='Furthermore, food plays a significant role in fostering global cultural experiences and connections with a diverse range of individuals.'),
 Document(id='4d82f67a-929a-45ec-9817-b65d61976b9b', metadata={'date': '2023-01-01', 'author': 'Alice', 'source': 'doc1'}, page_conte

In [8]:
### async
await vectorstore.asimilarity_search("What is the dog")

[Document(id='ad6ac9d9-a0bc-4d83-98dc-3fb9342c3f38', metadata={'date': '2023-05-01', 'author': 'Bob', 'source': 'doc5'}, page_content='The cat is a domestic animal. Its scientific name is Felis catus. It is a small animal that belongs to the “Felidae” family.'),
 Document(id='504e67b6-828f-4d0e-88b0-f03d1a8daa95', metadata={'source': 'doc4', 'author': 'Alice', 'date': '2023-04-01'}, page_content='Cats are one of the most popular and cherished pets worldwide. Known for their playful nature, independence, and affectionate demeanour, cats have been human companions for centuries. '),
 Document(id='bf38f0ce-a7ef-40de-a042-6291814a17d2', metadata={'author': 'Charlie', 'date': '2023-03-01', 'source': 'doc3'}, page_content='Furthermore, food plays a significant role in fostering global cultural experiences and connections with a diverse range of individuals.'),
 Document(id='4d82f67a-929a-45ec-9817-b65d61976b9b', metadata={'date': '2023-01-01', 'source': 'doc1', 'author': 'Alice'}, page_conte

In [9]:
vectorstore.similarity_search_with_score("What is the cat family called")

[(Document(id='ad6ac9d9-a0bc-4d83-98dc-3fb9342c3f38', metadata={'source': 'doc5', 'date': '2023-05-01', 'author': 'Bob'}, page_content='The cat is a domestic animal. Its scientific name is Felis catus. It is a small animal that belongs to the “Felidae” family.'),
  0.6697719097137451),
 (Document(id='504e67b6-828f-4d0e-88b0-f03d1a8daa95', metadata={'source': 'doc4', 'date': '2023-04-01', 'author': 'Alice'}, page_content='Cats are one of the most popular and cherished pets worldwide. Known for their playful nature, independence, and affectionate demeanour, cats have been human companions for centuries. '),
  1.0357447862625122),
 (Document(id='bf38f0ce-a7ef-40de-a042-6291814a17d2', metadata={'author': 'Charlie', 'source': 'doc3', 'date': '2023-03-01'}, page_content='Furthermore, food plays a significant role in fostering global cultural experiences and connections with a diverse range of individuals.'),
  1.8387198448181152),
 (Document(id='baa13407-b3c8-4321-8ecb-29b8d2f71bd1', metadat

In [11]:
# Retrievers
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k = 1)
retriever.batch(["cat", "dog", "food"])


[[Document(id='ad6ac9d9-a0bc-4d83-98dc-3fb9342c3f38', metadata={'author': 'Bob', 'source': 'doc5', 'date': '2023-05-01'}, page_content='The cat is a domestic animal. Its scientific name is Felis catus. It is a small animal that belongs to the “Felidae” family.')],
 [Document(id='ad6ac9d9-a0bc-4d83-98dc-3fb9342c3f38', metadata={'date': '2023-05-01', 'source': 'doc5', 'author': 'Bob'}, page_content='The cat is a domestic animal. Its scientific name is Felis catus. It is a small animal that belongs to the “Felidae” family.')],
 [Document(id='4d82f67a-929a-45ec-9817-b65d61976b9b', metadata={'source': 'doc1', 'date': '2023-01-01', 'author': 'Alice'}, page_content='Food is vital to our survival and keeps our bodies in a functioning state. Food becomes a necessary component of our body’s functioning the moment we are born')]]

In [13]:
retriever = vectorstore.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k":1}
)
retriever.batch(["cat", "dog", "food"])

[[Document(id='ad6ac9d9-a0bc-4d83-98dc-3fb9342c3f38', metadata={'author': 'Bob', 'source': 'doc5', 'date': '2023-05-01'}, page_content='The cat is a domestic animal. Its scientific name is Felis catus. It is a small animal that belongs to the “Felidae” family.')],
 [Document(id='ad6ac9d9-a0bc-4d83-98dc-3fb9342c3f38', metadata={'source': 'doc5', 'date': '2023-05-01', 'author': 'Bob'}, page_content='The cat is a domestic animal. Its scientific name is Felis catus. It is a small animal that belongs to the “Felidae” family.')],
 [Document(id='4d82f67a-929a-45ec-9817-b65d61976b9b', metadata={'source': 'doc1', 'author': 'Alice', 'date': '2023-01-01'}, page_content='Food is vital to our survival and keeps our bodies in a functioning state. Food becomes a necessary component of our body’s functioning the moment we are born')]]

In [15]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

message = """ Answer the question based on the context provided. If you don't know the answer, just say that you don't know, don't try to make up an answer.
question: {question}
context: {context} """

prompt = ChatPromptTemplate.from_messages([
    ("human" , message)
])
 
rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm

In [17]:
rag_chain.batch([" what is cat", "what is dog", "How is the food"])

[AIMessage(content='According to the provided context, a cat is a domestic animal.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 144, 'total_tokens': 158, 'completion_time': 0.018376112, 'prompt_time': 0.017207289, 'queue_time': 0.266799642, 'total_time': 0.035583401}, 'model_name': 'Llama3-8b-8192', 'system_fingerprint': 'fp_0fb809dba3', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--a1eb32bb-633e-44a1-abd1-e380e673e040-0', usage_metadata={'input_tokens': 144, 'output_tokens': 14, 'total_tokens': 158}),
 AIMessage(content="I don't know. The provided context is about cats, not dogs.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 149, 'total_tokens': 165, 'completion_time': 0.020381806, 'prompt_time': 0.069927206, 'queue_time': 0.422419665, 'total_time': 0.090309012}, 'model_name': 'Llama3-8b-8192', 'system_fingerprint': 'fp_bc17983608', 'servic